In [11]:
#API et packages
import pandas as pd 
import requests
from faker import Faker 
import random 
from mistralai import Mistral 
import os 
import time 
import re 
from concurrent.futures import ThreadPoolExecutor, as_completed
#start_time = time.perf_counter()
#end_time = time.perf_counter()
#elapsed_time = (end_time - start_time)/60
#print(elapsed_time)
os.environ["MISTRAL_API_KEY"]="4n72jJx0kSjjxYwU7KP24xpEaRg2EAq3"
print("Clé API définie")

#initialisation 
api_key= os.getenv("MISTRAL_API_KEY") 
if not api_key: 
    raise ValueError("Clé API manquante. Ajoutez une clé valide pour Mistral.") 
print("Clé API récupérée") 
mistral = Mistral(api_key=api_key)
print("Objet Mistral initialisé") 

Clé API définie
Clé API récupérée
Objet Mistral initialisé


In [12]:
#test API
import os
from mistralai import Mistral

api_key = os.environ["MISTRAL_API_KEY"]
model = "mistral-large-latest"

client = Mistral(api_key=api_key)

chat_response = client.chat.complete(
    model=model,
    messages=[
        {
            "role": "user",
            "content": "Parle moi de théorie des firmes",
        },
    ],
    max_tokens=1000
)


print(chat_response.choices[0].message.content)

La théorie des firmes est une branche de l'économie qui étudie la nature, le comportement et la structure des entreprises. Elle cherche à comprendre pourquoi les entreprises existent, comment elles prennent des décisions, et comment elles interagissent avec le marché et d'autres entreprises. Voici quelques concepts clés et approches dans la théorie des firmes :

### 1. **Pourquoi les entreprises existent ?**
Ronald Coase, dans son article "The Nature of the Firm" (1937), a proposé que les entreprises existent pour minimiser les coûts de transaction. Les coûts de transaction incluent les coûts liés à la recherche d'informations, la négociation de contrats, et la surveillance de leur exécution. En internalisant certaines transactions, les entreprises peuvent réduire ces coûts par rapport à ce qu'ils seraient sur le marché libre.

### 2. **Théorie de l'agence**
Cette théorie examine les relations contractuelles entre les "principaux" (comme les actionnaires) et les "agents" (comme les dir

In [15]:
#création du modèle de pricing 
#profil utilisateur= type de voiture (a,b,c,d,e,f,j,s,m)+ nombre de kilomètres par an + expérience de conduite(age-obtention du permis)+ âge +degré d'automatisation(par tranche de 10%)

#on génère un dataset pour faire notre modèle de pricing: 

#génération des types de voitures 
catégories=['A','B','C','D','E','F','J','M','S']
#A: mini-citadines, B: petites voitures, C:voitures compactes, D:grosses voitures, E:voitures de prestige, F:voitures de luxe, J:sport uility vehicule, M:grandes voitures familiales et S:voitures de sport
liste_catégorie=[]
while len(liste_catégorie)!=5000: 
    cat=random.randrange(0,9) 
    liste_catégorie.append(catégories[cat])
    
#génération des nombres de kilomètres par an 

#source:https://fr.statista.com/statistiques/899204/trajet-parcouru-en-voiture-sur-les-douze-derniers-mois-voiture-france/
#étant donné que les données sont sous la forme d'intervalle on va prendre en compte la médiane et les 3% qui restent sont répartis dans les autres catégories 
#31%=5000km; 55%=12500; 12%=37500; 2%=62500
import random

def generate_distances(total_count, percentages, distances):
    """
    Génère une liste de distances en fonction des pourcentages spécifiés.

    :param total_count: Nombre total de distances à générer.
    :param percentages: Liste des pourcentages correspondant à chaque distance.
    :param distances: Liste des distances correspondant à chaque pourcentage.
    :return: Liste de distances générées.
    """
    distance_list = []

    for percentage, distance in zip(percentages, distances):
        count = int(total_count * percentage / 100)
        distance_list.extend([distance] * count)
    while len(distance_list) < total_count:
        distance_list.append(distances[0]) 
    random.shuffle(distance_list)

    return distance_list

total_count = 5000  # Nombre total de distances à générer
percentages = [31, 55, 12, 2]
distances = [5000, 12500, 37500, 62500]

liste_distance = generate_distances(total_count, percentages, distances)

#génération de l'expérience de conduite 
liste_expérience=[]
liste_age=[]
while len(liste_expérience)!=5000:
    age=random.randrange(18,100)
    permis=random.randrange(18,100)
    if age>permis:
        expérience=age-permis 
        liste_expérience.append(expérience)    
        liste_age.append(age)

#génération du département
liste_département=[]
while len(liste_département)!=5000: 
    dép=random.randrange(1,97) 
    liste_département.append(dép)
    

#print des listes 
print(liste_catégorie)
print(liste_distance)
print(liste_age)
print(liste_expérience)
print(liste_département)

pricing_df = pd.DataFrame({'type de voiture':liste_catégorie,'expérience (en année)':liste_expérience,'age':liste_age,'distance parcouru par an (en kilomètres)':liste_distance, 'département':liste_département})
pricing_df.to_csv('pricing.csv')

['E', 'F', 'E', 'D', 'F', 'S', 'S', 'J', 'E', 'A', 'J', 'E', 'B', 'F', 'M', 'J', 'F', 'D', 'E', 'J', 'C', 'S', 'M', 'J', 'M', 'B', 'S', 'B', 'M', 'J', 'A', 'M', 'A', 'M', 'C', 'C', 'S', 'D', 'S', 'E', 'D', 'M', 'D', 'B', 'M', 'F', 'S', 'C', 'C', 'J', 'J', 'F', 'C', 'A', 'D', 'S', 'F', 'A', 'C', 'B', 'F', 'F', 'D', 'A', 'F', 'F', 'J', 'A', 'C', 'B', 'F', 'B', 'F', 'E', 'S', 'S', 'C', 'C', 'M', 'D', 'D', 'E', 'D', 'F', 'D', 'E', 'J', 'C', 'M', 'J', 'S', 'J', 'M', 'M', 'E', 'B', 'D', 'S', 'M', 'F', 'E', 'A', 'D', 'S', 'J', 'A', 'E', 'C', 'A', 'J', 'E', 'A', 'M', 'J', 'M', 'F', 'D', 'M', 'C', 'E', 'E', 'A', 'C', 'J', 'J', 'J', 'B', 'E', 'D', 'D', 'S', 'M', 'C', 'F', 'B', 'C', 'F', 'C', 'A', 'B', 'S', 'F', 'J', 'S', 'C', 'S', 'C', 'S', 'S', 'B', 'D', 'B', 'E', 'E', 'B', 'J', 'E', 'F', 'M', 'F', 'F', 'F', 'D', 'C', 'D', 'M', 'C', 'S', 'D', 'F', 'J', 'S', 'D', 'B', 'E', 'A', 'S', 'A', 'J', 'B', 'B', 'E', 'F', 'A', 'S', 'F', 'S', 'M', 'A', 'A', 'S', 'C', 'A', 'M', 'M', 'F', 'M', 'D', 'D', 'C',

In [13]:
automatisation=['0%','10%','20%','30%','40%','50%','60%','70%','80%','90%','100%']
liste_automatisation=[]
while len(liste_automatisation)!=5000: 
    cat=random.randrange(0,11) 
    liste_automatisation.append(automatisation[cat])
liste_automatisation

['10%',
 '30%',
 '0%',
 '20%',
 '0%',
 '100%',
 '10%',
 '20%',
 '100%',
 '50%',
 '30%',
 '10%',
 '70%',
 '30%',
 '20%',
 '10%',
 '30%',
 '20%',
 '30%',
 '90%',
 '90%',
 '40%',
 '10%',
 '30%',
 '80%',
 '50%',
 '100%',
 '30%',
 '100%',
 '90%',
 '20%',
 '70%',
 '0%',
 '40%',
 '100%',
 '90%',
 '20%',
 '80%',
 '60%',
 '10%',
 '40%',
 '60%',
 '20%',
 '20%',
 '30%',
 '0%',
 '50%',
 '20%',
 '30%',
 '20%',
 '20%',
 '90%',
 '60%',
 '20%',
 '30%',
 '40%',
 '30%',
 '20%',
 '40%',
 '40%',
 '70%',
 '0%',
 '40%',
 '80%',
 '0%',
 '0%',
 '40%',
 '40%',
 '50%',
 '10%',
 '20%',
 '100%',
 '100%',
 '80%',
 '80%',
 '50%',
 '80%',
 '100%',
 '80%',
 '40%',
 '50%',
 '60%',
 '40%',
 '50%',
 '0%',
 '60%',
 '40%',
 '30%',
 '0%',
 '90%',
 '50%',
 '50%',
 '40%',
 '0%',
 '60%',
 '50%',
 '10%',
 '60%',
 '90%',
 '10%',
 '70%',
 '30%',
 '20%',
 '30%',
 '0%',
 '60%',
 '70%',
 '90%',
 '90%',
 '20%',
 '100%',
 '90%',
 '30%',
 '0%',
 '40%',
 '10%',
 '40%',
 '40%',
 '10%',
 '70%',
 '100%',
 '10%',
 '0%',
 '80%',
 '20%',
 '2

In [16]:
pricing2_df = pd.DataFrame({
    'type de voiture': liste_catégorie,
    'expérience (en année)': liste_expérience,
    'age':liste_age,
    'distance parcourue par an (en kilomètres)': liste_distance,
    'département': liste_département,
    "degré d'automatisation de la conduite" : liste_automatisation
})
pricing2_df.to_csv('pricing2.csv', index=False)

In [17]:
#prix assurance2
answer = []

for i in range(len(pricing2_df)): 
    if i < 0 or i >= len(pricing2_df):
        print(f"Indice invalide : {i}")
        continue  

    prompt_content = f"Donne un prix précis (sous la forme suivante : x €) pour une assurance automobile tous risques en fonction des paramètres suivants sans donner d'explications : {pricing2_df.iloc[i].to_dict()}"
    try:
        chat_response = client.chat.complete(
            model=model,
            messages=[
                {
                    "role": "user",
                    "content": prompt_content,
                },
            ],
            max_tokens=50,
            temperature=1
        )

        response_content = chat_response.choices[0].message.content.strip()
        print(response_content)
        time.sleep(2)  # Pause pour éviter un dépassement du quota API
        answer.append(response_content)

    except Exception as e:
        print(f"Erreur lors de la requête pour l'indice {i} : {e}")
        continue

#nettoyage des réponses 
answer2 = [x[-8:] for x in answer] 
answer3 = [x.replace(".", "").replace(" ", "").replace("€", "") for x in answer2] 

#dataframe
pricing3_df = pd.DataFrame({
    'type de voiture': liste_catégorie,
    'expérience (en année)': liste_expérience,
    'age':liste_age,
    'distance parcourue par an (en kilomètres)': liste_distance,
    'département': liste_département,
    'prix assurance': answer3,
    'automatisation': liste_automatisation
})
pricing3_df.to_csv('pricing3.csv', index=False)


600 €
1050 €
450 €
750 €
1250 €
650 €
590 €
1500 €
750 €
980 €
850 €
750 €
1200 €
1200 €
1200 €
950 €
650 €
650 €
950 €
850 €
800 €
650 €
1450 €
1200 €
850 €
950 €
1200 €
650 €
2150 €
680 €
600 €
950 €
450 €
950 €
1200 €
950 €
2500 €
1800 €
750 €
650 €
1200 €
1200 €
1200 €
750 €
650 €
700 €
650 €
752 €
1250 €
1200 €
950 €
Je ne peux pas fournir un prix précis pour une assurance automobile tous risques en fonction des paramètres donnés sans accès à une base de données ou à un outil de tarification d'une
1200 €
900 €
850 €
1200 €
650 €
Je ne peux pas fournir un prix précis pour une assurance automobile tous risques, car cela dépend de nombreux facteurs spécifiques et varie en fonction des compagnies d'assurance
850 €
850 €
800 €
1200 €
1200 €
680 €
750 €
689 €
1200 €
765 €
1200 €
1215 €
750 €
1200 €
1200 €
2 100 €
1200 €
850 €
550 €
1200 €
845 €
1500 €
950 €
750 €
650 €
650 €
1050 €
750 €
2500 €
750 €
800 €
600 €
874 €
450 €
150 €
950 €
Je ne peux pas donner un prix précis pour une assur